# Examples of Use and Graphs:

In [1]:
import yaml
import torch
import copy
import pandas as pd

## If You're Just Looking for the Graphs!

These can be found in the notebooks:

- `graph_synthetic.ipynb`
- `graph_ecg.ipynb`
- `graph_cifarn_different_noise.ipynb`

## Loading the Optimisers:

The key part of this research is the optimisers, modified to perform Loss Adapted Plasticity (LAP) training. These have been implemented in Pytorch by modifying the Adam and SGD classes. These modified versions can be imported using:

In [2]:
from loss_adapted_plasticity import LAP

Documentation for both has been updated to reflect the modifications:

In [3]:
help(LAP)

Help on class LAP in module loss_adapted_plasticity.lap_wrapper:

class LAP(builtins.object)
 |  LAP(optimizer: torch.optim.optimizer.Optimizer, lap_n: int = 10, depression_strength: float = 1.0, depression_function='discrete_ranking_std', depression_function_kwargs: dict = {}, source_is_bool: bool = False, **opt_kwargs)
 |  
 |  Methods defined here:
 |  
 |  __getattr__(self, name)
 |  
 |  __getstate__(self)
 |      # defined since __getattr__ causes pickling problems
 |  
 |  __init__(self, optimizer: torch.optim.optimizer.Optimizer, lap_n: int = 10, depression_strength: float = 1.0, depression_function='discrete_ranking_std', depression_function_kwargs: dict = {}, source_is_bool: bool = False, **opt_kwargs)
 |      Depression won't be applied until at least :code:`lap_n` loss values
 |      have been collected for at least two sources. This could be 
 |      longer if a :code:`hold_off` parameter is used in the depression function.
 |      
 |      This class will wrap any optimis

These optimisers can be used in exactly the same way that you have previously used other pytorch optimisers, except for the fact that they take two extra arguments in the step method.

In current code, the optimisation process looks as follows:

```python
inputs, labels = data_batch
# ======= forward ======= 
outputs = model(inputs)
loss = criterion(outputs, labels)
# ======= backward =======
loss.backward()
optimizer.step()
```

When using a LAP optimiser, this needs to be changed to (with the key difference being the optimiser step ```optimizer.step(loss, source)```):

```python
inputs, labels = data_batch
# ======= forward ======= 
outputs = model(inputs)
loss = criterion(outputs, labels)
# ======= backward =======
loss.backward()
optimizer.step(loss, source)
```


To wrap any optimiser, use:

In [8]:
example_model = torch.nn.Linear(100,10)

optimizer = LAP(
    torch.optim.Adam, 
    params=example_model.parameters(), 
    lr=0.01,
    lap_n=25,
    depression_strength=1.0,
    # below is passed as dict as different depression functions can easily be addded
    depression_function_kwargs={'strictness': 0.8} 
    )

## Ready to Fit Models:

Two models have already been set up with the use of these optimisers built in. To access these, please use the following:

In [4]:
from experiment_code.models.model_code.mlp import MLPLearning
from experiment_code.models.model_code.conv3net import Conv3NetLearning
from experiment_code.models.model_code.resnet import ResNetLearning

The optimiser can be chosen in the following way:

In [5]:
# MLP with adam_lap optimizer
mlp = MLPLearning(
    train_optimizer={
        'Adam_lap': {
            'params': ['all'],
            'lr': 0.01,
            'lap_n': 20,
            'depression_function': 'discrete_ranking_std',
            'depression_function_kwargs': {},
            'depression_strength': 1.0,
            }
        }
    )

# Conv3Net with sgd_lap
cn = Conv3NetLearning(
    train_optimizer={
        'sgd_lap': {
            'params': ['all'],
            'lr': 0.01,
            'momentum': 0.9,
            'lap_n': 20,
            'depression_function': 'discrete_ranking_std',
            'depression_function_kwargs': {},
            'depression_strength': 1.0,
            }
        }
    )

# ResNet with adam_lap
rn = ResNetLearning(
    train_optimizer={
        'adam_lap': {
            'params': ['all'],
            'lr': 0.01,
            'lap_n': 20,
            'depression_function': 'discrete_ranking_std',
            'depression_function_kwargs': {},
            'depression_strength': 1.0,
            }
        }
    )

In the above, changing the ```depression_strength``` value from ```1.0``` to ```0.0``` will regain the standard model training and not apply LAP. Also, `sgd_lap` and `adam_lap` are already implemented through strings. If you want to use a different base optimiser, such as `Adagrad`, then you can specify it using: `'Adagrad_lap'`. An example is given for the MLP model above, with `'Adam_lap'`.

Or different models can be loaded using the ```synthetic_config.yaml``` file by specifying a model name.

In the following example, we will load and fit the ```Conv3Net-c_lbf-drstd``` model. This is a convolutional network made for training on CIFAR-10 with label flipping corruption applied to the data. For an explanation of ```c_[CODE]``` codes, see the ```README.MD``` file.

In [6]:
# the following loads the model config for the model Conv3Net-c_lbf-drstd
model_config = yaml.load(
    open('./synthetic_config.yaml', 'r'),
    Loader=yaml.FullLoader
    )['Conv3Net-c_lbf-drstd']
model_config

{'model_name': 'Conv3Net-c_lbf-drstd',
 'model_params': {'input_dim': 32,
  'in_channels': 3,
  'channels': 32,
  'n_out': 10,
  'n_epochs': 25,
  'train_optimizer': {'adam_lap': {'params': ['all'],
    'lr': 0.001,
    'lap_n': 20,
    'depression_strength': 1.0,
    'depression_function': 'discrete_ranking_std',
    'depression_function_kwargs': {'strictness': 0.8, 'hold_off': 0}}},
  'train_criterion': 'CE'},
 'train_params': {'train_method': 'traditional source',
  'source_fit': True,
  'n_sources': 10,
  'source_size': 128,
  'n_corrupt_sources': 4,
  'corruption_function': 'label_flip',
  'corruption_function_kwargs': {'source_save': True},
  'validation': {'do_val': True, 'train_split': 0.75, 'corrupt': False}},
 'save_params': {'model_path': './outputs/models/',
  'result_path': './outputs/results/'}}

We make a model config for ```depression_strength``` equal to ```1.0``` and ```0.0```, so that we can compare them.

In [7]:
# LAP Model
model_config_ds1 = copy.deepcopy(model_config)

# Standard Model
model_config_ds0 = copy.deepcopy(model_config)
(model_config_ds0
    ['model_params']
    ['train_optimizer']
    ['adam_lap']
    ['depression_strength']
    ) = 0.0

This config can then be used to load the model class and to build the model with the ```get_model``` function:

In [8]:
from experiment_code.models.model_code.utils import get_model

# LAP Model
model_class_ds1 = get_model(model_config_ds1)
model_ds1 = model_class_ds1(verbose=True, 
                            model_name=model_config_ds1['model_name'],
                            **model_config_ds1['model_params'],
                            **model_config_ds1['save_params']
                            )

# Standard Model
model_class_ds0 = get_model(model_config_ds0)
model_ds0 = model_class_ds0(verbose=True, 
                            model_name=model_config_ds0['model_name'],
                            **model_config_ds0['model_params'],
                            **model_config_ds0['save_params']
                            )

Given a training dataloader, this model can be easily fitted using the ```fit``` method. Training data can also be loaded using the ```model_config```, and ```args``` from ```argparse```. If using a notebook, you might want to use ```ArgFake``` to produce an ```args``` type object that can be used in replacement of ```argparse```.

In [9]:
from experiment_code.utils.utils import ArgFake

In [10]:
args = ArgFake({

    'dataset_name': 'cifar10',
    'verbose': True,
    'data_dir': './data/',
    'seed': 2,

})

The ```train_params``` part of the model config contains the information on how to corrupt the data, whilst ```args``` contains the dataset name and data directory.

In [11]:
model_config_ds1['train_params']

{'train_method': 'traditional source',
 'source_fit': True,
 'n_sources': 10,
 'source_size': 128,
 'n_corrupt_sources': 4,
 'corruption_function': 'label_flip',
 'corruption_function_kwargs': {'source_save': True},
 'validation': {'do_val': True, 'train_split': 0.75, 'corrupt': False}}

Data can be loaded with the ```get_train_data``` function.

In [12]:
from experiment_code.data_utils.dataloader_loaders import get_train_data

# data loaders are the same for model_config_ds1 and model_config_ds0
train_loaders = get_train_data(args, model_config_ds1)

Files already downloaded and verified
Files already downloaded and verified
 --------- Producing the data loaders --------- 


This has produced a list of data loaders containing both the training and validation data:

In [13]:
train_loaders

These can be used to fit the model:

In [14]:
# LAP Model
model_ds1.fit(train_loader=train_loaders[0], val_loader=train_loaders[1])

Training:    [##########]       Epoch: 25        Took: 1.7e+01s      Loss: 2.58e+00    Val Loss: 1.11e+00      Acc: 49.3%        Val Acc: 63.1%   Train Took: 4.4e+02s          


Conv3NetLearning(
  (net): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (relu1): ReLU()
    (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (relu2): ReLU()
    (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (relu3): ReLU()
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pm_fc): Sequential(
    (fc1): Linear(in_features=1024, out_features=64, bias=True)
    (relu1): ReLU()
  )
  (pm_clf): Linear(in_features=64, out_features=10, bias=True)
  (train_criterion): CrossEntropyLoss()
)

In [15]:
# Standard Model
model_ds0.fit(train_loader=train_loaders[0], val_loader=train_loaders[1])

Training:    [##########]       Epoch: 25        Took: 1.7e+01s      Loss: 1.82e+00    Val Loss: 1.56e+00      Acc: 40.6%        Val Acc: 49.5%   Train Took: 4.3e+02s          


Conv3NetLearning(
  (net): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (relu1): ReLU()
    (mp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (relu2): ReLU()
    (mp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (relu3): ReLU()
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pm_fc): Sequential(
    (fc1): Linear(in_features=1024, out_features=64, bias=True)
    (relu1): ReLU()
  )
  (pm_clf): Linear(in_features=64, out_features=10, bias=True)
  (train_criterion): CrossEntropyLoss()
)

Testing the model's performance on the CIFAR-10 test data is similar. We use the test config, as well as a set of arguments to load the test data:

In [16]:
from experiment_code.data_utils.dataloader_loaders import get_test_data

In [17]:
args = ArgFake({

    'dataset_name': 'cifar10',
    'test_method': 'traditional',
    'data_dir': './data/',
    'verbose': True,
    'seed': 2,

})

In [18]:
# a test config is loaded to ensure all testing is performed in the same way
test_config = yaml.load(
    open('./synthetic_config.yaml', 'r'),
    Loader=yaml.FullLoader
    )['testing-procedures'][args.test_method]

In [19]:
# these configs are passed to the get_test_data function to load the data
test_loader, test_targets = get_test_data(args, test_config=test_config)

Files already downloaded and verified
Files already downloaded and verified
 --------- Producing the data loaders --------- 


The predictions can then be made by using the following method:

In [20]:
outputs_ds1 = model_ds1.predict(test_loader=test_loader, targets_too=True)

Predicting:  [####################]       Batch: 50          Took: 0.00     Predict Took: 1.8e+00s          


In [21]:
confidence, predictions = outputs_ds1.max(dim=1)
accuracy = torch.sum(predictions == test_targets)/len(test_targets)
print('Accuracy with LAP was {:.2f}%'.format(accuracy*100))

Accuracy with LAP was 63.20%


In [22]:
outputs_ds0 = model_ds0.predict(test_loader=test_loader, targets_too=True)

Predicting:  [####################]       Batch: 50          Took: 0.00     Predict Took: 1.8e+00s          


In [23]:
confidence, predictions = outputs_ds0.max(dim=1)
accuracy = torch.sum(predictions == test_targets)/len(test_targets)
print('Accuracy with the standard model was {:.2f}%'.format(accuracy*100))

Accuracy with the standard model was 50.03%


## Graphs:

Running the code specified in the ```README.MD``` will save models in the `outputs/models/` directory, training loss graphs in the `outputs/results/` directory, test results in the `outputs/synthetic_results/` directory, and graphs in the `outputs/graphs/`.

The below is an exmaple of the graphs produced:

In [26]:
from IPython.display import Image
Image(url="./outputs/graphs/accuracy_results_complete.png", width=600,)

After running the code in the `graphs_[EXPERIMENT NAME].ipynb` notebooks, the `outputs/graphs/` directory will update with the graphs